# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con = engine)
X = df['message']
y = df.iloc[:,4:]
category_names = y.columns.values

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer() 

    clean_tokens = []
    for tok in tokens:    
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() 
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#Using multioutputclassifier with RandomForestClassifier as estimator
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('rfc', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#Splitting X an Y
X_train, X_test, Y_train, Y_test = train_test_split(X, y)
model = pipeline

In [6]:
#Trainning model
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
#Saving column names in target value
target = y.columns.values.astype('str')

In [8]:
#Predicting
y_pred = model.predict(X_test) 
print(classification_report(Y_test.values, y_pred, target_names=target))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      4952
               request       0.85      0.40      0.54      1122
                 offer       0.00      0.00      0.00        37
           aid_related       0.75      0.52      0.62      2705
          medical_help       0.62      0.06      0.11       515
      medical_products       0.63      0.06      0.11       302
     search_and_rescue       0.50      0.03      0.06       192
              security       0.50      0.01      0.02       111
              military       0.53      0.05      0.08       221
                 water       0.80      0.33      0.47       422
                  food       0.81      0.31      0.44       745
               shelter       0.85      0.23      0.36       564
              clothing       0.50      0.05      0.09        96
                 money       0.83      0.04      0.07       140
        missing_people       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
#Looking for model parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f72e6669620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('rfc',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
#Using gridsearch to tune the model
parameters = {  
    'rfc__estimator__max_depth': [None, 5, 10],
    'rfc__estimator__max_features': ['sqrt','log2']
} 

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)
y_pred_test = cv.predict(X_test)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
print(classification_report(Y_test.values, y_pred_test, target_names=target))

                        precision    recall  f1-score   support

               related       0.81      0.94      0.87      4952
               request       0.81      0.38      0.52      1122
                 offer       0.00      0.00      0.00        37
           aid_related       0.74      0.51      0.60      2705
          medical_help       0.58      0.09      0.15       515
      medical_products       0.68      0.09      0.16       302
     search_and_rescue       0.67      0.03      0.06       192
              security       0.50      0.01      0.02       111
              military       0.64      0.06      0.12       221
                 water       0.85      0.25      0.38       422
                  food       0.84      0.32      0.46       745
               shelter       0.89      0.19      0.31       564
              clothing       0.43      0.03      0.06        96
                 money       1.00      0.03      0.06       140
        missing_people       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
#Using DecisionTreeClassifier as estimator
pipeline_v2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('dtc', MultiOutputClassifier(DecisionTreeClassifier()))])

In [13]:
#f1-score results are better using DecisionTreeClassifier rather than RandomForestClassifier.
#DecisionTreeClassifier will be used for the rest of the project
model_v2 = pipeline_v2
model_v2.fit(X_train, Y_train)
y_pred_test_v2 = model_v2.predict(X_test)
print(classification_report(Y_test.values, y_pred_test_v2, target_names=target))

                        precision    recall  f1-score   support

               related       0.83      0.86      0.85      4952
               request       0.58      0.58      0.58      1122
                 offer       0.05      0.03      0.03        37
           aid_related       0.64      0.63      0.63      2705
          medical_help       0.36      0.31      0.33       515
      medical_products       0.37      0.36      0.36       302
     search_and_rescue       0.28      0.18      0.22       192
              security       0.09      0.09      0.09       111
              military       0.36      0.38      0.37       221
                 water       0.65      0.66      0.65       422
                  food       0.72      0.69      0.71       745
               shelter       0.61      0.60      0.60       564
              clothing       0.54      0.48      0.51        96
                 money       0.32      0.30      0.31       140
        missing_people       0.24      

### 9. Export your model as a pickle file

In [14]:
#Exporting model
import pickle
pickle.dump(model_v2, open('model.pkl','wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.